In [9]:
import os
import scipy
import scipy.cluster
import sklearn
import sklearn.cluster
import numpy
from PIL import Image
from PIL import ImageColor

ModuleNotFoundError: No module named 'scipy'

In [13]:
def write(json_data: dict, file_name: str):
    with open(file_name, 'w') as f:
        f.write(json.dumps(json_data))

In [14]:
# Download list of all currencies
import urllib
import json

url = 'https://api.coingecko.com/api/v3/coins/list?include_platform=true'
response = urllib.request.urlopen(url)
currencies = json.loads(response.read())
write(currencies, 'tmp_step_01.json')
print('Downloaded list of ', len(currencies))

Downloaded list of  12951


In [16]:
# Include only native or ethereum ERC20s

def include(currency: dict) -> bool:
    if len(currency['platforms']) == 0:
        return True
    if 'ethereum' in currency['platforms']:
        return len(currency['platforms']['ethereum']) != 0
    return False

currencies = list(filter(lambda c: include(c) , currencies))
write(currencies, 'tmp_step_02.json')
print('Filtered out unsupported, remaining count', len(currencies))

Filtered out unsupported, remaining count 6301


In [12]:
# Get image url rank description

base_url = 'https://api.coingecko.com/api/v3/coins/'
query_params = '?localization=false&tickers=false&market_data=false&community_data=false&developer_data=false&sparkline=false'
processed_currencies = list()

for currency in currencies:
    url = base_url + currency['id'] + query_params
    response = urllib.request.urlopen(url)
    info = json.loads(response.read())
    currency['description'] = info['description']['en']
    currency['link'] = info['links']['homepage'][0]
    currency['twitter'] = info['links']['homepage'][0]

id
symbol
name
platforms
description en
links homepage 1st
links twitter_screen_name
image large
coingecko_rank
